In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=fefefb636660fcc88cfe3a97ac2ee61300e96fceff1062e2f89c55c3ef913cf6
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [2]:
with open(file='/kaggle/input/the-jungle-book/jungleb.txt', encoding='utf-8', mode='r') as input_fp:
    text = input_fp.read()
len(text)

292391

In [3]:
from nltk.tokenize import sent_tokenize
import pandas as pd

df = pd.DataFrame(data={'text': sent_tokenize(text=text.replace('\n', ' '), language='english')})
df['text'] = df['text'].apply(func=lambda x: ' '.join(x.split()))
df = df.drop_duplicates(ignore_index=True)
df['char count'] = df['text'].str.len()
df['word count'] = df['text'].apply(func=lambda x: len(x.split()))
df['text40'] = df['text'].str[:40]
df = df.sample(frac=1.0, random_state=2023)
df.shape

(2579, 4)

We are going to leave all the Project Gutenberg additions to the text in place.

In [4]:
from plotly.express import scatter
scatter(data_frame=df.sort_values(ascending=True, by='word count').reset_index(), y=['word count', 'char count'] )

In [5]:
from arrow import now
from sentence_transformers import SentenceTransformer
MODEL = 'all-MiniLM-L6-v2'

model_start = now()
model = SentenceTransformer(model_name_or_path=MODEL,)
model.max_seq_length = 256
print('model max sequence length: {}'.format(model.max_seq_length))
embeddings = model.encode(df['text'].values.tolist())
print('model time: {}'.format(now() - model_start))

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

model max sequence length: 256


Batches:   0%|          | 0/81 [00:00<?, ?it/s]

model time: 0:00:30.656281


In [6]:
from sentence_transformers import util

similarity_start = now()
cos_sim = util.cos_sim(embeddings, embeddings)

# todo use a comprehension here
all_sentence_combinations = []
for i in range(len(cos_sim)-1):
    if (i > 0 and i % 200 == 0) or (i == len(df) - 1):
        print('{}: {}'.format(now(), i))
    for j in range(i+1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

print('{}: calculated similarities.'.format(now()))
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print('similarity time: {}'.format(now() - similarity_start))

2023-12-21T19:34:33.982944+00:00: 200
2023-12-21T19:34:37.839209+00:00: 400
2023-12-21T19:34:40.867479+00:00: 600
2023-12-21T19:34:43.677314+00:00: 800
2023-12-21T19:34:46.493594+00:00: 1000
2023-12-21T19:34:48.174818+00:00: 1200
2023-12-21T19:34:50.899028+00:00: 1400
2023-12-21T19:34:52.061009+00:00: 1600
2023-12-21T19:34:53.100262+00:00: 1800
2023-12-21T19:34:53.798846+00:00: 2000
2023-12-21T19:34:54.295561+00:00: 2200
2023-12-21T19:34:56.226651+00:00: 2400
2023-12-21T19:34:56.314685+00:00: calculated similarities.
similarity time: 0:04:42.844953


In [7]:
for score, i, j in all_sentence_combinations[:10]:
    print('cosine: {:.4f} indices: {} {} text: {}  text: {}'.format(cos_sim[i][j], i, j, df['text'].values[i][:40], df['text'].values[j][:40], ))

cosine: 1.0000 indices: 886 2108 text: You may copy it, give it away or re-use   text: You may copy it, give it away or re-use 
cosine: 0.9776 indices: 407 2501 text: All things made he--Shiva the Preserver.  text: _All things made he--Shiva the Preserver
cosine: 0.9463 indices: 287 1999 text: Little Toomai stared again and again.  text: Little Toomai stared once more.
cosine: 0.9316 indices: 650 1683 text: For a young king cobra?  text: For a young cobra?
cosine: 0.9099 indices: 474 2027 text: “Ha!  text: “Hah!
cosine: 0.9082 indices: 286 422 text: Section 2.  text: Section 1.
cosine: 0.9006 indices: 979 1307 text: “Good.  text: “Good!
cosine: 0.8983 indices: 334 1561 text: Project Gutenberg™ eBooks are often crea  text: Project Gutenberg eBooks may be modified
cosine: 0.8972 indices: 1003 2559 text: None the less, Two Tails is a great cowa  text: Two Tails is a great coward.
cosine: 0.8959 indices: 1498 2517 text: “A man’s cub.  text: “I speak for the man’s cub.


In [8]:
embeddings.shape

(2579, 384)

In [9]:
from sklearn.manifold import TSNE
tsne_start = now()
tsne = TSNE(n_components=2, n_iter=10000, verbose=1, init='pca', random_state=2023)
df[['t0', 't1',]] = tsne.fit_transform(X=embeddings)
print('TSNE time: {}'.format(now() - tsne_start))

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 2579 samples in 0.001s...
[t-SNE] Computed neighbors for 2579 samples in 0.356s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2579
[t-SNE] Computed conditional probabilities for sample 2000 / 2579
[t-SNE] Computed conditional probabilities for sample 2579 / 2579
[t-SNE] Mean sigma: 0.295464
[t-SNE] KL divergence after 250 iterations with early exaggeration: 79.832230
[t-SNE] KL divergence after 10000 iterations: 1.778032
TSNE time: 0:02:30.719885


In [10]:
from plotly.express import scatter
scatter(data_frame=df, x='t0', y='t1', hover_name='text40', color=df['text'].apply(func=lambda x: 'gutenberg' not in x.lower()))

Obviously we have a distinct Gutenberg cluster that is isolated; we would expect this because the Project Gutenberg boilerplate is written in a different style than the rest of the text.

Let's use a vectorizer to pick out particularly important words and see if the embeddings capture their sentences as being similar.

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

def find_term(arg:str, terms: list):
    lower = arg.lower().split()
    for term in terms:
        if term in lower:
            return term
    return 'none'

vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words='english', max_df=0.5, min_df=0.001)
fit_result = vectorizer.fit_transform(raw_documents=df['text'].values)
word_weight_df = pd.DataFrame(data={'word': vectorizer.get_feature_names_out(), 'weight': fit_result.toarray().sum(axis=0),}).sort_values(ascending=False, by='weight')
top_terms = [word for word in  word_weight_df.head(n=20)['word'].values.tolist() if word not in {'head', 'come', 'time', 'thou', 'big', 'know', 'like', 'said', 'good', 'people'}]
top_terms = top_terms + ['elephant', 'cub', 'rikki-tikki', 'khan', 'akela', 'cobra', 'kala',  'kotick', 'teddy'] # let's add in some others
print('top terms: {}'.format(sorted(top_terms)))
plot_df = df.copy()
plot_df['term'] = plot_df['text'].apply(func=find_term, args=(top_terms, ))

top terms: ['akela', 'bagheera', 'baloo', 'cobra', 'cub', 'elephant', 'jungle', 'kala', 'khan', 'kotick', 'little', 'man', 'mowgli', 'nag', 'rikki', 'rikki-tikki', 'teddy', 'tikki', 'wolf']


In [12]:
from plotly.colors import qualitative
scatter(data_frame=plot_df, x='t0', y='t1', hover_name='text40', color='term', color_discrete_sequence=qualitative.Light24, height=800)

We do have sentences that are easily understood as being part of clusters based on their nouns, particularly when these are proper nouns. In other cases we have sentences that are too undistinguished - being too short or having only pronouns for nouns - that our model can't do much to cluster them.